In [1]:
import datetime
from glob import glob
import json
import os
import sys
import yaml
from tqdm import tqdm
import argparse

import numpy as np
import h5py
from PIL import Image
import resampy
import torch
from torchvision import transforms
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
from scipy.io.wavfile import write

# sys.path.append("../..")
# import hifigan
# from hifigan.env import AttrDict
# from hifigan.models import Generator

# sys.path.append("../U2S")
# # import U2S
# from hparams import create_hparams
# from train import load_model
# from text import text_to_sequence

sys.path.append("./models")
# from models import models_modified
from models import TransformerConditionedLM
from models_modified import TransformerSentenceLM_FixedImg, TransformerSentenceLM_FixedImg_gated

2023-04-03 15:12:24.938912: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-03 15:12:34.326799: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-04-03 15:12:34.328160: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-04-03 15:12:34.328167: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
# from utils_i2u import *
from utils_synthesize import *
from judge_asr import judge_ans

sys.path.append('../')
from gslm.unit2speech.tts_data import (
    TacotronInputDataset,
)
from gslm.unit2speech.utils import (
    load_quantized_audio_from_file,
    load_tacotron,
    load_waveglow,
    synthesize_audio,
)

In [3]:
with open('../../config.yml') as yml:
        config = yaml.safe_load(yml)
    
global device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
model_path = "../../saved_model/I2U/VC_5_captions_224_hubert/23-03-20_19:17:08_uLM_sentence"
word_map_path="../../data/processed/VC_5_captions_224_hubert/WORDMAP_coco_5_cap_per_img_1_min_word_freq.json"

config_path = glob(model_path + "/config*.yml")[0]
# config_path = glob(model_path+"/*")
model_checkpoint = glob(model_path+"/*BEST*.tar")[0]
# word_map_path="../../data/processed/SpokenCOCO_LibriSpeech/WORDMAP_coco_1_cap_per_img_1_min_word_freq.json"

# Load word map (word2ix)
global word_map, rev_word_map, special_words
with open(word_map_path) as j:
    word_map = json.load(j)
rev_word_map = {v: k for k, v in word_map.items()}  # ix2word
special_words = {"<unk>", "<start>", "<end>", "<pad>"}

with open(config_path, 'r') as yml:
    model_config = yaml.safe_load(yml)

model_params = model_config["i2u"]["model_params"]
model_params['vocab_size'] = len(word_map)
model_params['refine_encoder_params'] = model_config["i2u"]["refine_encoder_params"]

from models import PositionalEncoding
model = TransformerSentenceLM_FixedImg(**model_params)
model.pos_encoder = PositionalEncoding(d_model=1024, max_len=152)
model.load_state_dict(torch.load(model_checkpoint)["model_state_dict"])
model.to(device)

TransformerSentenceLM_FixedImg(
  (embed): Embedding(104, 1024)
  (pos_encoder): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (LM_decoder): None
  (classifier): Linear(in_features=1024, out_features=104, bias=True)
  (image_encoder): DinoResEncoder_Raw(
    (resnet): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (4): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track

In [ ]:
# """
#     Trained U2S tacotron model on VC data
# """
# tts_model_path = "/net/papilio/storage2/yhaoyuan/transformer_I2S/gslm_models/u2S/HuBERT_KM100_tts_checkpoint_best.pt"
# max_decoder_steps = 500
# code_dict_path = "/net/papilio/storage2/yhaoyuan/transformer_I2S/gslm_models/u2S/HuBERT_KM100_code_dict"

# # hifigan_checkpoint_path = config["u2s"]['hifigan']
# hifigan_checkpoint_path = "../../hifigan/LJ_FT_T2_V3/generator_v3"
# asr_checkpoint_path = config["asr"]["model_path"]

# # tacotron_model = load_tacotron(tacotron_checkpoint_path, tacotron_max_decoder_step)
# tacotron_model, tts_datasets = load_tacotron2_hubert(model_path=tts_model_path, code_dict_path=code_dict_path, max_decoder_steps=max_decoder_steps)
# hifigan_model = load_hifigan(hifigan_checkpoint_path, device)
# asr_model, asr_processor = load_asr(asr_checkpoint_path, device)

In [6]:
"""
    Pretrained U2S tacotron model
"""
tacotron_max_decoder_step = 500
tacotron_checkpoint_path = "../../saved_model/U2S/outdir_VC_hubert_22050_102_warm/checkpoint_33000"
hifigan_checkpoint_path = "../../hifigan/LJ_FT_T2_V3/generator_v3"
asr_checkpoint_path = config["asr"]["model_path"]

tacotron_model = load_tacotron2(tacotron_checkpoint_path, tacotron_max_decoder_step, vocab_size=102)
hifigan_model = load_hifigan(hifigan_checkpoint_path, device)
asr_model, asr_processor = load_asr(asr_checkpoint_path, device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Removing weight norm...


In [7]:
def load_images(data_path, split):
    image_hdf5 = glob(data_path+f"/{split}*.hdf5")[0]
    image_names = glob(data_path+f"/{split}*.json")[0]
    h = h5py.File(image_hdf5, 'r')
    images = h['images']
    with open(image_names, "r") as f:
        names = json.load(f)
    return images, names

def get_transformed_img(img, transform): # -> torch.tensor
    # img = img.transpose(2, 0, 1) # (224, 224, 3) -> (3, 224, 224)
    img = torch.FloatTensor(img / 255.)
    if transform is not None:
        img = transform(img)
    return img.to(device)

In [8]:
image_data_path = "../../data/RL/224"
split = "TEST"
imgs, names = load_images(image_data_path, split)
names = names[:10]
transform = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

In [19]:
# for i in tqdm(range(len(names)), desc=f"Getting {split} Results"):
for i in range(len(names)):
    img = get_transformed_img(imgs[i], transform)
    # img = get_transformed_img(imgs[i], transform=None)
    img = img.unsqueeze(0)
    name = names[i]
    seqs = model.decode(image=img, start_unit=word_map["<start>"], end_unit=word_map["<end>"], max_len=150, beam_size=10)
    
    words = seq2words(seq=seqs, rev_word_map=rev_word_map, special_words=special_words)
    audio = u2s(
        words=words,
        tacotron2_model=tacotron_model,
        hifigan_model=hifigan_model,
        device=device
        )
    # audio = u2s_hubert(
    #     words=words,
    #     tacotron2_model=tacotron_model,
    #     tts_dataset=tts_datasets,
    #     hifigan_model=hifigan_model,
    #     device=device
    #     )

    # audio_display = audio*32768.0
    # audio_display = audio_display.astype('int16')
    import IPython.display as ipd
    display(ipd.Audio(audio, rate=22050))
    
    trans = s2t(audio=audio, asr_model=asr_model, asr_processor=asr_processor, device=device)
    break
    print("\n")

In [20]:
trans

'there is one eggplant in a white background'